In [2]:
from pyomo.environ import *

In [3]:
# m = ConcreteModel()
# m.d1 = Var(within=NonNegativeReals, bounds=(0,5))
# m.d2 = Var(within=NonNegativeReals, bounds=(0,5))
# 
# F_t = 0.9
# 
# m.c1 = Constraint(expr=(1/16)*(3*m.d1 - 3*m.d2 + 10) >= F_t)
# m.c2 = Constraint(expr=(1/11)*(-3*m.d1 + 6*m.d2 + 10) >= F_t)
# 
# m.obj = Objective(expr=10*m.d1 + 10*m.d2, sense=minimize)
# 
# SolverFactory('gurobi').solve(m, tee=True)

In [4]:
# for c in m.component_objects(Constraint, active=True):
#     print(f"\nConstraint: {c.name}")
#     for i in c:
#         con = c[i]
#         body_val = value(con.body)
#         if con.has_ub():
#             slack = value(con.upper) - body_val
#             print(f"Slack (UB): {slack}")
#         if con.has_lb():
#             slack = body_val - value(con.lower)
#             print(f"Slack (LB): {slack}")


In [25]:
model = ConcreteModel()

# Design variables
model.x = Var(bounds=(0, 4))
model.y = Var(bounds=(0, 3))

# Binary variables indicating region membership
model.delta1 = Var(within=Binary)
model.delta2 = Var(within=Binary)

M = 1000
target = 0.65

# Region 1: x in [0,2], y in [0,2]
model.r1x = Constraint(expr=model.x <= 2 + M*(1 - model.delta1))
model.r1y = Constraint(expr=model.y <= 2 + M*(1 - model.delta1))
model.r1x_lb = Constraint(expr=model.x >= 0 - M*(1 - model.delta1))
model.r1y_lb = Constraint(expr=model.y >= 0 - M*(1 - model.delta1))

# Region 2: x in (2,4], y in [1,3]
model.r2x = Constraint(expr=model.x >= 2 + 1e-4 - M*(1 - model.delta2))  # Open lower bound
model.r2x_ub = Constraint(expr=model.x <= 4 + M*(1 - model.delta2))
model.r2y = Constraint(expr=model.y >= 1 - M*(1 - model.delta2))
model.r2y_ub = Constraint(expr=model.y <= 3 + M*(1 - model.delta2))

# Only one region active
model.only_one_region = Constraint(expr=model.delta1 + model.delta2 == 1)

# SF expressions
sf1 = 2*model.x + 3*model.y
sf2 = 4*model.x + model.y

# Conditional constraints
model.sf1_cons = Constraint(expr=sf1 >= target - M*(1 - model.delta1))
model.sf2_cons = Constraint(expr=sf2 >= target - M*(1 - model.delta2))

model.obj = Objective(expr=10*model.x + 20*model.y, sense=minimize)

SolverFactory('gurobi').solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': 3.2499999999998863, 'Upper bound': 3.2499999999998863, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 4, 'Number of binary variables': 2, 'Number of integer variables': 2, 'Number of continuous variables': 2, 'Number of nonzeros': 24, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0010001659393310547', 'Error rc': 0, 'Time': 0.17607426643371582}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [26]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 10*x + 20*y


In [27]:
model.x.pprint()

x : Size=1, Index=None
    Key  : Lower : Value               : Upper : Fixed : Stale : Domain
    None :     0 : 0.32499999999998863 :     4 : False : False :  Reals


In [28]:
model.y.pprint()

y : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  -0.0 :     3 : False : False :  Reals


In [29]:
model.delta1.pprint()

delta1 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   1.0 :     1 : False : False : Binary


In [30]:
model.delta2.pprint()

delta2 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :     1 : False : False : Binary


In [31]:
import numpy as np

In [37]:
p,w = np.polynomial.legendre.leggauss(1)

In [38]:
p

array([0.])

In [39]:
w

array([2.])